In [16]:
import openai  # for making OpenAI API requests
from datasets import load_dataset
import tiktoken
import numpy as np
import re

openai.api_key = 'API KEY'

In [27]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Encoding name	OpenAI models

# cl100k_base:	gpt-4, gpt-3.5-turbo, text-embedding-ada-002
# p50k_base:	text-davinci-003, text-davinci-002
# r50k_base:	GPT-3 models (text-curie-001, text-babbage-001, text-ada-001, davinci, curie, babbage, ada)

#print(num_tokens_from_string("Hello world, let's test tiktoken.", "cl100k_base"))

In [28]:
dataset = load_dataset("gsm8k", 'main')

In [36]:
thinking_styles = np.array([
    'Let’s think step by step.',  # Original
    'Let’s make a step by step plan and implement it with good notion and explanation',  # Original
    'How could I devise an experiment to help solve the problem?',  # Original
    'How can I simplify the problem so that it is easier to solve?',  # Original
    'Try creative thinking, generate innovative and out-of-the-box ideas to solve the problem. Explore unconventional solutions, thinking beyond traditional boundaries, and encouraging imagination and originality.',  # Original
    'Critically evaluate the given information. Are there any biases or assumptions that might be influencing the interpretation?',  # Critical Thinking
    'Imagine a similar situation in a different time or place. How might it have been approached historically?',  # Historical Thinking
    'Project the outcomes into the future. If we follow this path, where might we end up in 10 years?',  # Futuristic Thinking
    'Put yourself in the shoes of all the stakeholders involved. How would they feel about this situation?',  # Empathetic Thinking
    'Consider how the different elements of this problem interact with one another. Can we identify any feedback loops or systemic issues?',  # Systems Thinking
    'Reflect on the ethical dimensions of this challenge. Are there moral principles that should guide our decision-making?',  # Ethical Thinking
    'Let’s compare this situation to a similar one in a different context. Are there lessons we can draw from that comparison?',  # Comparative Thinking
    'Consider the problem holistically. How do all the pieces fit together in the larger context?',  # Holistic Thinking
    'Trust your gut feeling on this. What does your intuition tell you about the best approach?',  # Intuitive Thinking
    'Examine the conflicting viewpoints around this issue. Can we find a middle ground or synthesis?',  # Dialectical Thinking
])

prompt_instruction = np.array([
    'Solve the math word problem, giving your answer as an arabic numeral',  
    'Solve the problem by explaining why systemic or structural issues would not be the cause of the issue.', 
    'Generate the answer to a word problem and write it as a number.',  
    'Break down the math problem step-by-step and provide the final numeric solution.', 
    'Apply fundamental mathematical principles to solve the problem, and provide a clear numeric answer.',
    'Use visual aids like diagrams or graphs if necessary to better illustrate the process of solving the problem.',
    'Consider using an algorithm or formula that is most efficient in solving this type of math problem. Provide the final answer as a number.',
    'Discuss the rationale behind each step you take in solving the problem, ensuring the explanation is clear and concise.',
    'If applicable, use real-world examples to contextualize the math problem, and solve it providing a numeric answer.',
    'Detail the mathematical concepts or theories utilized in solving the problem, and conclude with a numeric solution.',
    'Present multiple methods to solve the problem if possible, concluding each with a clear numeric answer.',
    'Ensure accuracy by double-checking calculations and logic used in solving the problem, and present the final numeric solution.',
    'Adapt a teaching approach, explaining the problem and solution as if educating someone with basic mathematical knowledge.',
])

prompt_mutators = np.array([
    'Generate a prompt mutant that introduces an element of suspense or intrigue.',  
    'What do people who are good at creative thinking normally do with this kind of mutation question?',  
    'Change the following instruction being as brief as possible',  
    'Modify the prompt to emphasize the importance of collaboration or teamwork in the process.',
    'Rework the instruction to appeal to someone with a background in arts or humanities.',
    'Twist the prompt to challenge someone with advanced expertise in the subject.',
    'Adjust the instruction to encourage out-of-the-box thinking or a novel approach.',
    'Refine the prompt to make it sound like a riddle or puzzle to be solved.',
    'Adapt the original instruction to cater to a younger audience or beginners.',
    'Convert the prompt to make it sound like a quest or mission.',
    'Reimagine the instruction as if it is a call to action for a global challenge or competition.',
    'Infuse humor or sarcasm into the original prompt.'
])


In [37]:

def create_prompt_examples(indexes, dataset):

    prompt_examples_str = 'These are some examples of similar problems and how to solve them: \n'

    for index in indexes:

        prompt_examples_str = prompt_examples_str + 'Q: ' + dataset['train'][int(index)]['question'] + '\nA: ' + dataset['train'][int(index)]['answer'] + '\n'

    return prompt_examples_str


def create_population(thinking_styles, instructions, prompt_examples, dataset, test_indexes):

    population =  []

    problems = dataset['test'][test_indexes]['question']

    answers = dataset['test'][test_indexes]['answer']

    for i in range(len(instructions)):

        prompt = thinking_styles[i] + '\n' + instructions[i] + '\n' + prompt_examples + '\nQ: ' + problems[i]

        population.append(prompt)


    return population, answers

def mutate_instructions1(instructions, mutation_prompts, indexes_to_mutate, indexes_good_prompts):

    instructions_for_mutation, new_instructions = [], np.copy(instructions)

    if len(indexes_good_prompts) == 0:

        for i in range(len(indexes_to_mutate)):

            selected_instruction_index = indexes_to_mutate[np.random.randint(len(indexes_to_mutate))]

            instructions_for_mutation.append(mutation_prompts[np.random.randint(3)] + 
                                             '. The prompt you need to change is the following: \n' + 
                                             instructions[selected_instruction_index])


    elif len(indexes_good_prompts) > 0:

        for i in range(len(indexes_to_mutate)):

            selected_instruction_index = indexes_good_prompts[np.random.randint(len(indexes_good_prompts))]

            instructions_for_mutation.append(mutation_prompts[np.random.randint(3)] + ': \n' + instructions[selected_instruction_index])
    
    
    response = openai.Completion.create(model="gpt-3.5-turbo-instruct", prompt=instructions_for_mutation, max_tokens=2000)

    for choice in response.choices: # type: ignore

        new_instructions[indexes_to_mutate[choice.index]] = choice.text

    return new_instructions


def mutate_instructions(instructions, mutation_prompts, indexes_to_mutate, indexes_good_prompts):

    instructions_for_mutation, new_instructions = [], np.copy(instructions)

    if len(indexes_good_prompts) == 0:

        for i in range(len(indexes_to_mutate)):

            selected_instruction_index = indexes_to_mutate[np.random.randint(len(indexes_to_mutate))]

            instructions_for_mutation.append(mutation_prompts[np.random.randint(3)] + 
                                             '. The prompt you need to change is the following: \n' + 
                                             instructions[selected_instruction_index])


    elif len(indexes_good_prompts) > 0:

        for i in range(len(indexes_to_mutate)):

            selected_instruction_index = indexes_good_prompts[np.random.randint(len(indexes_good_prompts))]

            instructions_for_mutation.append(mutation_prompts[np.random.randint(3)] + ': \n' + instructions[selected_instruction_index])
    
    
    response = openai.Completion.create(model="gpt-3.5-turbo-instruct", prompt=instructions_for_mutation, max_tokens=2000)

    for choice in response.choices: # type: ignore

        new_instructions[indexes_to_mutate[choice.index]] = choice.text

    return new_instructions


def last_number(s):
    numbers = re.findall(r'\d+', s)
    return int(numbers[-1]) if numbers else None


def evaluate(prompts, answers):

    response = openai.Completion.create(model="gpt-3.5-turbo-instruct", prompt=prompts, max_tokens=2000)

    model_ans = ['']*len(answers)

    evaluations = np.zeros(len(answers))

    for choice in response.choices: # type: ignore

        model_ans[choice.index] = choice.text

        if last_number(answers[choice.index]) == last_number(choice.text):

            evaluations[choice.index] = 1
    
    return evaluations, model_ans


In [38]:
## INITIALIZATION

prompt_examples_indexes = np.random.randint(100, size=3)

initial_thinking_styles = thinking_styles[np.random.randint(15, size=5)]

initial_prompt_instruction = prompt_instruction[np.random.randint(12, size=5)]

initial_prompt_mutators = prompt_mutators[np.random.randint(11, size=5)]

prompt_examples = create_prompt_examples(prompt_examples_indexes, dataset)

test_problem_indexes = np.random.randint(100, size=5)

initial_population, initial_answers = create_population(initial_thinking_styles, initial_prompt_instruction, prompt_examples, dataset, test_problem_indexes)

num_generations = 7

In [47]:
print('Individual example:')
print()
print(initial_population[0])

Individual example:

Let’s compare this situation to a similar one in a different context. Are there lessons we can draw from that comparison?
Solve the math word problem, giving your answer as an arabic numeral
These are some examples of similar problems and how to solve them: 
Q: An earthquake caused four buildings to collapse. Experts predicted that each following earthquake would have double the number of collapsing buildings as the previous one, since each one would make the foundations less stable. After three more earthquakes, how many buildings had collapsed including those from the first earthquake?
A: The second earthquake caused 2 * 4 = <<2*4=8>>8 buildings to collapse.
The third earthquake caused 2 * 8 = <<2*8=16>>16 buildings to collapse.
The fourth earthquake caused 16 * 2 = <<16*2=32>>32 buildings to collapse.
Including the first earthquake, the earthquakes caused 4 + 8 + 16 + 32 = <<4+8+16+32=60>>60 buildings to collapse.
#### 60
Q: Shawna's workout goal is 30 situps. O

In [40]:
# ALGORITHM
#------------------------------------------------------------------------------
#------------------------------------------------------------------------------

population = initial_population.copy()
answers = initial_answers.copy()
mutators = initial_prompt_mutators.copy()
thinking_styles = initial_thinking_styles.copy()
prompt_instructions = initial_prompt_instruction.copy()

for i in range(num_generations):
    
    evaluations, model_ans = evaluate(population, answers)

    good_indexes = np.where(evaluations==1)[0]
    bad_indexes = np.where(evaluations==0)[0]

    prompt_instructions = mutate_instructions(prompt_instructions, mutators, bad_indexes, good_indexes)

    population, answers = create_population(thinking_styles, prompt_instructions, prompt_examples, dataset, test_problem_indexes)

    population_fitness = np.mean(evaluations)

    print('Mean Generation', i, 'Fitness: ', population_fitness)
            

Generation 0 Fitness:  0.6
Generation 1 Fitness:  0.6
Generation 2 Fitness:  0.6
Generation 3 Fitness:  0.6
Generation 4 Fitness:  0.8
Generation 5 Fitness:  0.6
Generation 6 Fitness:  0.8


In [48]:
print('Final Population Scores: ', evaluations)
print('-------------------------------------')
print()
print('Final Population Instructions: ') 
print()
print(prompt_instructions)
print('-------------------------------------')
print()
print('final Answers:')
model_ans

Final Population Scores:  [0. 1. 1. 1. 1.]
-------------------------------------

Final Population Instructions: 

[' However, instead of using traditional arithmetic operations, try to find creative or unconventional ways to solve the problem.\n\n"Think o'
 ' ' 'Generate the answer to a word problem and write it as a number.'
 '\n\nCan you accurately predict the outcome or solution to this word problem? The number, when revealed, will leave you with a sense of myst'
 ' Additionally, think creatively and consider alternative methods or strategies to solve the problem.\n\n"To arrive at the solution for this']
-------------------------------------

final Answers:


["\nA: Jill's annual salary as a teacher is 20 * 50 * 35 = <<20*50*35=35000>>$35,000.\nHer annual salary as a coach is 30 * 50 * 15 = <<30*50*15=22500>>$22,500.\nTherefore, her total annual salary is 35,000 + 22,500 = <<35,000+22,500=57500>>$57,500.",
 '\n\nA: Each school has sent 2 teams of 5 players each, so 4 schools would have sent 4*2*5 = 40 players.\nEach school has also sent 2 coaches, so 4 schools would have sent 4*2 = 8 coaches.\nIn total, all of the schools would have sent 40 + 8 = 48 people.',
 '\nA: Alex weighs 4 * 125 - 2 = <<4*125-2=498>>498 pounds.\nTogether, Grace and Alex weigh 125 + 498 = <<125+498=623>>623 pounds.\n#### 623',
 '\nA: If Sarah does 400 pounds of laundry, then David does 400/4 = <<400/4=100>>100 pounds of laundry.\nSince Raymond does half as much as Sarah, he does 400/2 = <<400/2=200>>200 pounds of laundry.\nThe difference in the amount of laundry they do is 200 - 100 = <<200-100=100>>100 pounds.\n#### 100',
 '\n\nA: In the second 20 minutes, Mike score

In [45]:
num_tokens_from_string(model_ans[4], 'cl100k_base')

257

In [1]:
### test
a = [1,2,2]